In [1]:
from tensorflow.keras.layers import (Conv2D, UpSampling2D, Conv2DTranspose, concatenate, MaxPooling2D, GlobalAveragePooling2D, 
                                     Activation, Dropout, Cropping2D, Flatten, Dense, BatchNormalization)
from tensorflow.keras.models import Model
from tensorflow.keras import Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import InceptionV3
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
import efficientnet.tfkeras as efn

Using TensorFlow backend.


In [2]:
IMG_HEIGHT = 300
IMG_WIDTH = 300
TRAIN_IMAGE_PATH = "jpeg/train/"
TEST_IMAGE_PATH = "jpeg/test/"
BATCH_SIZE = 8
AUTO = tf.data.experimental.AUTOTUNE
REG = 0.0005
EPOCHS=100
METRICS = [
    tf.keras.metrics.TruePositives(name='tp'),
    tf.keras.metrics.FalsePositives(name='fp'),
    tf.keras.metrics.TrueNegatives(name='tn'),
    tf.keras.metrics.FalseNegatives(name='fn'), 
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
    tf.keras.metrics.AUC(name='auc'),
    tf.keras.metrics.binary_crossentropy,
]

In [3]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [4]:
train_df["image_name"] = train_df["image_name"].apply(lambda x: TRAIN_IMAGE_PATH + x + ".jpg")
test_df["image_name"] = test_df["image_name"].apply(lambda x: TEST_IMAGE_PATH + x + ".jpg")

In [5]:
train_df_target_1 = train_df[train_df["target"] == 1]
train_df_target_0 = train_df[train_df["target"] == 0]

In [6]:
del train_df

In [7]:
random_target_0 = np.random.randint(low=1, high=train_df_target_0.shape[0], 
                                    size=2 * train_df_target_1.shape[0])

In [8]:
train_df_d = pd.concat([train_df_target_0.iloc[random_target_0], train_df_target_1])

In [9]:
def decode_image(filename, label=None, image_size=(IMG_WIDTH, IMG_HEIGHT)):
    bits = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(bits, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, size=image_size)
    
    if label is None:
        return image
    else:
        return image, label
#image = tf.cast(image, tf.float32)/255.0
    
def data_augment(image, label=None):
    r_crop = np.random.uniform(low = 0.4, high = 1.0)
    r_rsize = np.random.uniform(low = 0.8, high = 1.2)
    image = tf.image.random_crop(image, (int(r_crop*IMG_HEIGHT), int(r_crop*IMG_WIDTH), 3))
    image = tf.image.rot90(image)
    image = tf.keras.preprocessing.image.random_shear(image, 20)
    image = tf.image.resize(image, (int(r_rsize*IMG_HEIGHT), int(r_rsize*IMG_WIDTH), 3), preserve_aspect_ratio=True)
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_saturation(image, lower=0.7, upper=1.3)
    image = tf.image.random_contrast(image, lower=0.7, upper=1.3)
    image = tf.image.random_brightness(image, lower=0.7, upper=1.3)
    image = tf.image.rgb_to_hsv(image)
    if label is None:
        return image
    else:
        return image, label
#     image = tf.image.rgb_to_hsv(image)
#     image = tf.image.random_flip_left_right(image)
#     image = tf.image.adjust_brightness(image, 0.2)
#     image = tf.image.rot90(image)
#     image = tf.image.central_crop(image, central_fraction=0.3)

In [10]:
x_train = train_df[["image_name"]]
y_train = train_df["target"].astype(np.float32).values
x_train.reset_index(drop=True, inplace=True)

In [11]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=45)

In [12]:
x_train.shape, x_val.shape, y_train.shape, y_val.shape

((1401, 1), (351, 1), (1401,), (351,))

In [13]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)

In [14]:
class_weights

array([0.74600639, 1.51623377])

In [15]:
train_dataset = (tf.data.Dataset
                 .from_tensor_slices((x_train["image_name"].values, y_train))
                 .map(decode_image, num_parallel_calls=AUTO)
                 .repeat()
                 .shuffle(512)
                 .batch(BATCH_SIZE)
                 .prefetch(AUTO)
                )

In [16]:
val_dataset = (tf.data.Dataset
               .from_tensor_slices((x_val["image_name"].values, y_val))
               .map(decode_image, num_parallel_calls=AUTO)
               .repeat()
               .shuffle(512)
               .batch(BATCH_SIZE)
               .prefetch(AUTO))

In [17]:
test_dataset = (tf.data.Dataset.from_tensor_slices((test_df.image_name))
                .map(decode_image, num_parallel_calls=AUTO)
                .cache()
                .batch(BATCH_SIZE))

In [18]:
lr = ReduceLROnPlateau(
    monitor="val_auc",
    patience=3,
    min_lr=0.000001,
    factor=0.5,
    verbose=1
)

In [19]:
es = EarlyStopping(monitor="val_loss", patience=10)

In [20]:
model_chkpt = ModelCheckpoint(filepath="best_model.h5")

In [21]:
def dice_loss(y_true, y_pred):
    numerator = 2 * tf.reduce_sum(y_true * y_pred, axis=-1)
    denominator = tf.reduce_sum(y_true + y_pred, axis=-1)
    return 1 - (numerator + 1) / (denominator + 1)

In [22]:
def create_model(input_layer):
    
    pretrained_model =InceptionV3(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
                                              weights='imagenet',
                                              include_top=False
                                             )
    # False = transfer learning, True = fine-tuning
    pretrained_model.trainable = True#False 

    inp1 = tf.keras.layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3), name='inp1')
        

    x=pretrained_model(inp1)
    x=tf.keras.layers.GlobalAveragePooling2D()(x)
    x=tf.keras.layers.Dense(1024, 
                                kernel_regularizer=tf.keras.regularizers.l2(l=0.01),
                                activation='relu')(x)
    x=tf.keras.layers.Dropout(0.2)(x)
    x=tf.keras.layers.Dense(512, 
                                kernel_regularizer=tf.keras.regularizers.l2(l=0.01),
                                activation='relu')(x)
    x=tf.keras.layers.Dropout(0.2)(x)
    x=tf.keras.layers.Dense(256, 
                                kernel_regularizer=tf.keras.regularizers.l2(l=0.01),
                                activation='relu')(x)
    x=tf.keras.layers.Dropout(0.2)(x)
    x=tf.keras.layers.Dense(128, 
                                kernel_regularizer=tf.keras.regularizers.l2(l=0.01),
                                activation='relu')(x)
    x=tf.keras.layers.Dropout(0.2)(x)
    x=tf.keras.layers.Dense(64, kernel_regularizer=tf.keras.regularizers.l2(l=0.01),
                                activation='relu')(x)
    x=tf.keras.layers.Dropout(0.2)(x)
        
    output = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(x)
        
    model = tf.keras.models.Model(inputs=[inp1], outputs=[output])
    
    model.compile(
        optimizer='adam',
        loss = dice_loss,
        metrics=[tf.keras.metrics.AUC()],
        )
        
    return model

In [23]:
input_layer = Input((IMG_HEIGHT, IMG_WIDTH, 3))
model = create_model(input_layer=input_layer)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inp1 (InputLayer)            [(None, 300, 300, 3)]     0         
_________________________________________________________________
inception_v3 (Model)         (None, 8, 8, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              2098176   
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0     

In [24]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss=dice_loss, metrics=METRICS)

In [25]:
history = model.fit(train_dataset, epochs=EPOCHS, callbacks=[lr, es, model_chkpt],
                        steps_per_epoch=x_train.shape[0]//BATCH_SIZE, validation_data=val_dataset,
                        validation_steps=x_val.shape[0]//BATCH_SIZE,
                        class_weight = class_weights)

Train for 175 steps, validate for 43 steps
Epoch 1/100
175/175 [==============================] - 197s 1s/step - loss: 18.0775 - tp: 15.0000 - fp: 2.0000 - tn: 933.0000 - fn: 450.0000 - accuracy: 0.6771 - precision: 0.8824 - recall: 0.0323 - auc: 0.7478 - binary_crossentropy: 0.8251 - val_loss: 11.2358 - val_tp: 53.0000 - val_fp: 8.0000 - val_tn: 222.0000 - val_fn: 61.0000 - val_accuracy: 0.7994 - val_precision: 0.8689 - val_recall: 0.4649 - val_auc: 0.8652 - val_binary_crossentropy: 0.6718
Epoch 2/100
175/175 [==============================] - 185s 1s/step - loss: 7.5123 - tp: 184.0000 - fp: 46.0000 - tn: 888.0000 - fn: 282.0000 - accuracy: 0.7657 - precision: 0.8000 - recall: 0.3948 - auc: 0.7853 - binary_crossentropy: 0.9267 - val_loss: 4.7305 - val_tp: 57.0000 - val_fp: 9.0000 - val_tn: 213.0000 - val_fn: 65.0000 - val_accuracy: 0.7849 - val_precision: 0.8636 - val_recall: 0.4672 - val_auc: 0.7827 - val_binary_crossentropy: 1.1641
Epoch 3/100
175/175 [==============================

175/175 [==============================] - 128s 730ms/step - loss: 0.1624 - tp: 422.0000 - fp: 12.0000 - tn: 928.0000 - fn: 38.0000 - accuracy: 0.9643 - precision: 0.9724 - recall: 0.9174 - auc: 0.9756 - binary_crossentropy: 0.2432 - val_loss: 0.2138 - val_tp: 74.0000 - val_fp: 28.0000 - val_tn: 204.0000 - val_fn: 38.0000 - val_accuracy: 0.8081 - val_precision: 0.7255 - val_recall: 0.6607 - val_auc: 0.8794 - val_binary_crossentropy: 0.5922
Epoch 17/100
175/175 [==============================] - 128s 731ms/step - loss: 0.1555 - tp: 420.0000 - fp: 7.0000 - tn: 941.0000 - fn: 32.0000 - accuracy: 0.9721 - precision: 0.9836 - recall: 0.9292 - auc: 0.9761 - binary_crossentropy: 0.2378 - val_loss: 0.2267 - val_tp: 79.0000 - val_fp: 24.0000 - val_tn: 193.0000 - val_fn: 48.0000 - val_accuracy: 0.7907 - val_precision: 0.7670 - val_recall: 0.6220 - val_auc: 0.8538 - val_binary_crossentropy: 0.7913
Epoch 18/100
174/175 [============================>.] - ETA: 0s - loss: 0.1544 - tp: 444.0000 - fp: 

175/175 [==============================] - 128s 731ms/step - loss: 0.1327 - tp: 441.0000 - fp: 6.0000 - tn: 931.0000 - fn: 22.0000 - accuracy: 0.9800 - precision: 0.9866 - recall: 0.9525 - auc: 0.9829 - binary_crossentropy: 0.1995 - val_loss: 0.2042 - val_tp: 77.0000 - val_fp: 29.0000 - val_tn: 198.0000 - val_fn: 40.0000 - val_accuracy: 0.7994 - val_precision: 0.7264 - val_recall: 0.6581 - val_auc: 0.8280 - val_binary_crossentropy: 0.7277
Epoch 33/100
175/175 [==============================] - 128s 731ms/step - loss: 0.1322 - tp: 453.0000 - fp: 3.0000 - tn: 922.0000 - fn: 22.0000 - accuracy: 0.9821 - precision: 0.9934 - recall: 0.9537 - auc: 0.9833 - binary_crossentropy: 0.2217 - val_loss: 0.1923 - val_tp: 84.0000 - val_fp: 26.0000 - val_tn: 202.0000 - val_fn: 32.0000 - val_accuracy: 0.8314 - val_precision: 0.7636 - val_recall: 0.7241 - val_auc: 0.8681 - val_binary_crossentropy: 0.6105
Epoch 34/100
175/175 [==============================] - 128s 734ms/step - loss: 0.1302 - tp: 445.0000

In [26]:
history

In [27]:
pred = model.predict(test_dataset)

In [28]:
pred

array([[1.3071077e-03],
       [1.7081249e-04],
       [5.6652707e-04],
       ...,
       [7.2037172e-01],
       [7.3512719e-04],
       [6.6319335e-01]], dtype=float32)

In [29]:
sam = pd.read_csv("sample_submission.csv")

In [30]:
sam.head()


,image_name,target
0,ISIC_0052060,0
1,ISIC_0052349,0
2,ISIC_0058510,0
3,ISIC_0073313,0
4,ISIC_0073502,0


In [31]:
sam["target"] = pred

In [32]:
sam

,image_name,target
0,ISIC_0052060,0.001307
1,ISIC_0052349,0.000171
2,ISIC_0058510,0.000567
3,ISIC_0073313,0.000642
4,ISIC_0073502,0.585236
...,...,...
10977,ISIC_9992485,0.000582
10978,ISIC_9996992,0.001894
10979,ISIC_9997917,0.720372
10980,ISIC_9998234,0.000735


In [33]:
sam.to_csv("inceptionv3_barzil_researcher_image_augmentation_exp.csv", index=False)